In [6]:
import requests
import json

# Specify the API endpoint
url = "https://hub.snapshot.org/graphql"

# Define the GraphQL query
query = """

query Proposals {
  proposals(
    first: 20,
    skip: 0,
    where: {
      space_in: ["polywrap.eth"],
      state: "closed"
    },
    orderBy: "created",
    orderDirection: desc
  ) {
    id
    title
    body
    choices
    start
    end
    snapshot
    state
    author
    space {
      id
      name
    }
  }
}
"""

# Define the headers for the request
headers = {'Content-Type': 'application/json'}

# Make the request
response = requests.post(url, json={'query': query}, headers=headers)

if response.status_code == 200:
    data = response.json()['data']['proposals']

    # Extract the proposal information
    proposals = []
    for proposal in data:
        proposals.append({
            "id": proposal['id'],
            "title": proposal['title'],
            "body": proposal['body'],
            "choices": proposal['choices'],
            "state": proposal['state'],
            "author": proposal['author'],
            "space": proposal['space']['name'],
            # Add any other fields you want to include
        })
else:
    print(f"Request failed with status code {response.status_code}")

proposals

[{'id': '0xd3dbc043148a1cd60534bec2748d6776aae01eabb35ea806987ca3a2f10bfc17',
  'title': 'CIDT / Macroscape – Service Agreement',
  'body': 'CIDT: [https://beta.fileverse.io/files/33467dd5-d910-4de5-a75a-8dd262fd7fb5](https://beta.fileverse.io/files/33467dd5-d910-4de5-a75a-8dd262fd7fb5)\n\nMacroscape (Nathan Windsor): [https://beta.fileverse.io/files/73821cae-4886-4aeb-8c57-3b531cb14a50](https://beta.fileverse.io/files/73821cae-4886-4aeb-8c57-3b531cb14a50)',
  'choices': ['Accept', 'Reject'],
  'state': 'closed',
  'author': '0x937F5b32Bc3cafcd1B02462F93e6AE5a843f6C6A',
  'space': 'Polywrap'},
 {'id': '0x7a53fed2fa8d3a1b59f5a08e6aebd316a1158847f299677c910324ed042494c5',
  'title': 'Alex Service Agreement',
  'body': 'https://beta.fileverse.io/files/3f9fc222-7373-4016-8d1a-c359277a1c14',
  'choices': ['Accept', 'Reject'],
  'state': 'closed',
  'author': '0x937F5b32Bc3cafcd1B02462F93e6AE5a843f6C6A',
  'space': 'Polywrap'},
 {'id': '0xabcb01928d4734fbbaf50c40a5a0d4c83dc2c66f930c22ea17acc

In [16]:
# Import the OpenAI library
import openai

# Define the initial system and user messages
initial_messages = [
    {"role": "system", "content": "You are a creative assistant that helps write blog post summaries."},
    {"role": "user", "content": "I have some information about a proposal on 'polywrap.eth' that I want to summarize into a blog post."},
]

# Open the file in write mode
with open("proposals-summary.md", "w") as file:

    # Loop through the proposals
    for proposal in proposals:
        # Create a copy of the initial messages
        messages = initial_messages.copy()
        
        # Create the user message for this proposal
        message = f"The proposal titled '{proposal['title']}' was authored by '{proposal['author']}'. The state of the proposal is '{proposal['state']}'. The details of the proposal are as follows: '{proposal['body']}'"
        
        # Add the user message to the messages
        messages.append({"role": "user", "content": message})

        # Generate the assistant's responses
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        # Write the assistant's response to the file
        file.write(response['choices'][0]['message']['content'])
        file.write("\n---\n")


In [19]:
# Import the OpenAI library
import openai

# Load the file content
with open("proposals-summary.md", "r") as file:
    file_content = file.read()

# Define the initial system and user messages
messages = [
    {"role": "system", "content": "You are a creative assistant that helps transform summaries into a detailed blog post newsletter."},
    {"role": "user", "content": f"I have these summaries that I want to transform into a blog post, remember to organize proposals into categories: {file_content}"},
]

# Generate the assistant's responses
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    max_tokens=3000
)

# Open the file and write the assistant's response
with open("proposals-newsletter.md", "w") as file:
    file.write(response['choices'][0]['message']['content'])
    file.write("\n---\n")


KeyboardInterrupt: 